In [13]:
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
spark = get_spark_session(dev=True)


In [14]:
test_namespace = 'gold' 
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}")

DataFrame[]

In [15]:
delivery_stats = spark.read.table(f"{test_namespace}.delivery_stats")
delivery_stats.show(n=5)

+--------------------+----------------------+----------------------+-----------------------+-------------------+-------+
|            order_id|lead_time_approve_days|lead_time_carrier_days|lead_time_customer_days|total_delivery_days|is_late|
+--------------------+----------------------+----------------------+-----------------------+-------------------+-------+
|a9a93c428c6103f21...|                     3|                     3|                     10|                 16|  false|
|56ef80c564f6fd57c...|                     0|                     0|                      7|                  7|  false|
|f9427374480e37251...|                     1|                     1|                      5|                  7|  false|
|d6d7c431275f0029d...|                     0|                     5|                      7|                 12|  false|
|0957ed870116e596b...|                     0|                     1|                      9|                 10|  false|
+--------------------+----------

In [16]:
target_product_purchase_order_info_with_segment = spark.read.table(f"{test_namespace}.target_product_purchase_order_info_with_segment")
target_product_purchase_order_info_with_segment.show(n=5)

+--------------------+--------------------+-------------+-------------------+-------------+--------------------+-----------+-------------+
|          product_id|            order_id|order_item_id|           purchase|     category|           seller_id|total_price|      segment|
+--------------------+--------------------+-------------+-------------------+-------------+--------------------+-----------+-------------+
|05f0fe07929d35be0...|362cd36939c1c4638...|            1|2018-06-25 23:00:42|health_beauty|c70c1b0d8ca86052f...|     118.35|Star Products|
|05f0fe07929d35be0...|b41cc5a0872a7fd88...|            1|2018-07-16 10:53:45|health_beauty|c70c1b0d8ca86052f...|     118.35|Star Products|
|05f0fe07929d35be0...|b71ba668b12da5320...|            1|2018-06-03 10:47:54|health_beauty|c70c1b0d8ca86052f...|     117.96|Star Products|
|05f0fe07929d35be0...|bcd881f346c66116b...|            2|2018-07-21 11:39:34|health_beauty|c70c1b0d8ca86052f...|     129.19|Star Products|
|05f0fe07929d35be0...|d1c3d

In [17]:
target_product_order_info = target_product_purchase_order_info_with_segment.select('order_id', 'order_item_id', 'product_id','category', 'seller_id','segment')
target_product_order_info.show()

+--------------------+-------------+--------------------+-------------+--------------------+--------------+
|            order_id|order_item_id|          product_id|     category|           seller_id|       segment|
+--------------------+-------------+--------------------+-------------+--------------------+--------------+
|362cd36939c1c4638...|            1|05f0fe07929d35be0...|health_beauty|c70c1b0d8ca86052f...| Star Products|
|b41cc5a0872a7fd88...|            1|05f0fe07929d35be0...|health_beauty|c70c1b0d8ca86052f...| Star Products|
|b71ba668b12da5320...|            1|05f0fe07929d35be0...|health_beauty|c70c1b0d8ca86052f...| Star Products|
|bcd881f346c66116b...|            2|05f0fe07929d35be0...|health_beauty|c70c1b0d8ca86052f...| Star Products|
|d1c3d911ad830cab7...|            2|05f0fe07929d35be0...|health_beauty|c70c1b0d8ca86052f...| Star Products|
|c8f3d770e7079f624...|            1|10a845be3ffe8a78d...|health_beauty|edb1ef5e36e0c8cd8...|    Niche Gems|
|0bbaf543aa84dd90b...|      

In [49]:
delivery_detail = target_product_order_info.join(delivery_stats, on='order_id', how='inner').orderBy('product_id', 'order_id')
delivery_detail.writeTo(f"{test_namespace}.delivery_detail").using('iceberg').createOrReplace()

In [50]:
delivery_detail.show()

+--------------------+-------------+--------------------+-------------+--------------------+--------------+----------------------+----------------------+-----------------------+-------------------+-------+
|            order_id|order_item_id|          product_id|     category|           seller_id|       segment|lead_time_approve_days|lead_time_carrier_days|lead_time_customer_days|total_delivery_days|is_late|
+--------------------+-------------+--------------------+-------------+--------------------+--------------+----------------------+----------------------+-----------------------+-------------------+-------+
|1fd6c29ecb9dd8b65...|            1|00210e41887c2a8ef...|health_beauty|e9779976487b77c6d...|Volume Drivers|                     0|                     1|                      1|                  2|  false|
|226975521c585d7bb...|            1|00210e41887c2a8ef...|health_beauty|e9779976487b77c6d...|Volume Drivers|                     0|                     1|                      8

In [ ]:
# long type
from pyspark.sql.functions import expr

delivery_detail_long = delivery_detail.select(
    "order_id",
    "product_id",
    expr("""
        stack(3,
            'approve', lead_time_approve_days,
            'carrier', lead_time_carrier_days,
            'customer', lead_time_customer_days
        ) as (lead_time_type, lead_time_days)
    """)
)
delivery_detail_long.show()

+--------------------+--------------------+--------------+--------------+
|            order_id|          product_id|lead_time_type|lead_time_days|
+--------------------+--------------------+--------------+--------------+
|1fd6c29ecb9dd8b65...|00210e41887c2a8ef...|       approve|             0|
|1fd6c29ecb9dd8b65...|00210e41887c2a8ef...|       carrier|             1|
|1fd6c29ecb9dd8b65...|00210e41887c2a8ef...|      customer|             1|
|226975521c585d7bb...|00210e41887c2a8ef...|       approve|             0|
|226975521c585d7bb...|00210e41887c2a8ef...|       carrier|             1|
|226975521c585d7bb...|00210e41887c2a8ef...|      customer|             8|
|2d8e71bf7d31a41d4...|00210e41887c2a8ef...|       approve|             2|
|2d8e71bf7d31a41d4...|00210e41887c2a8ef...|       carrier|             2|
|2d8e71bf7d31a41d4...|00210e41887c2a8ef...|      customer|             4|
|2d8e71bf7d31a41d4...|00210e41887c2a8ef...|       approve|             2|
|2d8e71bf7d31a41d4...|00210e41887c2a8e

In [ ]:
delivery_detail_long.writeTo(f"{test_namespace}.delivery_detail_long").using('iceberg').createOrReplace()

In [ ]:
spark.stop()